In [ ]:
# First initialize datasets. These are used for training and evaluating the model.

from pathlib import Path
from random import sample

from fortuna import ObjectDetectionDataset
from fortuna.data import pedestrians_dirpath
import fortuna.torchutils.transforms as T

# Specify paths to images and masks.
image_paths = sorted((pedestrians_dirpath / "Images").rglob("*"))
mask_paths = sorted((pedestrians_dirpath / "Masks").rglob("*"))

# We're going to use a random subsample of the dataset to validate the model.
n = 10 # len(image_paths)
training_fraction = 0.9
training_ix = sample(range(n), int(training_fraction*n))
validation_ix = list(set(range(n)) - set(training_ix))

# Create ObjectDetectionDatasets for training and validation.
training_dataset = ObjectDetectionDataset(
    image_paths=[image_paths[j] for j in training_ix],
    mask_paths=[mask_paths[j] for j in training_ix],
    transforms=[T.RandomHorizontalFlip(0.5)]
)
validation_dataset = ObjectDetectionDataset(
    image_paths=[image_paths[j] for j in validation_ix],
    mask_paths=[mask_paths[j] for j in validation_ix]
)

In [ ]:
# Initialize and fit the object detector.

from fortuna import ObjectDetectionModel
from fortuna.data import pedestrians_dirpath

model = ObjectDetectionModel(n_classes=2)

model.fit(
    training_dataset,
    validation_dataset
)

In [ ]:
# Evaluate the detector.

model.evaluate(
    validation_dataset
)

In [ ]:
# Export model.

model.write("path_to_model_file.onnx") # Export as e.g. ONNX model.